# Customer Satisfaction
### Business Problem Introduction
From frontline support teams to C-suites, customer satisfaction is a key measure of success. Unhappy customers don't stick around. What's more, unhappy customers rarely voice their dissatisfaction before leaving. Santander Bank is asking Kagglers to help them identify dissatisfied customers early in their relationship. Doing so would allow Santander to take proactive steps to improve a customer's happiness before it's too late.
In this competition, you'll work with hundreds of anonymized features to predict if a customer is satisfied or dissatisfied with their banking experience. The original dataset can be found on Kaggle: https://www.kaggle.com/c/santander-customer-satisfaction

### Quick Summary
Since the data is unbalanced, r_squared is not the best metric to be used. Therefore, we will use AUC (Area Under the Curve) as a metric:
* Decision tree is:  0.841
* Linear regression is:  0.7838
* Random forest AUC:  0.6872
* Logistic regression is:  0.5782

--------------
## View Train Data
The train data is the most important dataset in which we try to find trends on the TARGET column, Y indepdendent variable.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_validate
from statistics import mean
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.datasets import make_classification
from sklearn import ensemble
import sklearn.metrics as metrics

sample=r'/kaggle/input/santander-customer-satisfaction/sample_submission.csv'
train=r'/kaggle/input/santander-customer-satisfaction/train.csv'
test=r'/kaggle/input/santander-customer-satisfaction/test.csv'
#sample=r'C:\Users\sschm\Desktop\kaggle\customer\sample_submission.csv'
#train=r'C:\Users\sschm\Desktop\kaggle\customer\train.csv'
#test=r'C:\Users\sschm\Desktop\kaggle\customer\test.csv'

data=pd.read_csv(train)
data.drop_duplicates() #no duplicates
print(data.shape) #(76020, 371)
data.head() 

## Numeric DataFrame

In [2]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df = data.select_dtypes(include=numerics)
df.head()

## Find Missing Values
The dataset is still structured as (76020, 371) meaning all the columns must have been numeric. There are no missing values. 

In [3]:
#search for columns with missing values:
def findNA():
    print("Missing data by column as a percent:")
    findNA=df.isnull().sum().sort_values(ascending=False)/len(df)
    print(findNA.head())
findNA() 

## Examine Target Column
Customer satisfaction is in 1 and 0. Almost all of the TARGET column is 0 which is at 0.9604%.  Also, since the target values are only one and zero we can use logistic regression to further analyze any trends in the data. 

In [4]:
target=df['TARGET'].unique()
print(target)

#Check for unbalanced data:
targets=list(df['TARGET'])
zero = targets.count(0)
one = targets.count(1)
total=round(zero/len(targets),4)
print(total)

## Heatmap

In [5]:
def printHeat():
    corr = df.corr()
    #print(corr)
    y='TARGET'
    highly_corr_features = corr.index[abs(corr[y])>0.05]
    plt.figure(figsize=(10,10))
    heat = sns.heatmap(df[highly_corr_features].corr(),annot=True,cmap="RdYlGn")
    top10=corr[y].sort_values(ascending=False).head(10)
    print(heat)
    print("Top 10 Correlations:\n", top10) # top ten correlations
printHeat()

## Identify Highly Correlated Features:
First, we must remove highly correlated features that are above .80 correlation. Normally, creating a heapmap visualization is helpful but with 371 column features, will not be memory efficient unless using a supercomputer.

In [6]:
print(df.shape, " before removing highly correlated variables.")
# Create correlation matrix
corr_matrix = df.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

df=df.drop(to_drop, axis = 1)
print(df.shape, " after removing highly correlated variables.")

## Split the Data
Use 70% of the train data to predict the accuracy of the remaining 30% of the test data.

In [7]:
X=df.drop('TARGET', axis=1)
y=df['TARGET']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 42)

## Logistic Regression
AUC for logistic regression is:  0.5782.

In [8]:
from sklearn.linear_model import LogisticRegression
logReg = LogisticRegression(solver='liblinear') #solver param gets rid of encoder error

#Train the model and create predictions
logReg.fit(X_train, y_train)
logPredict = logReg.predict_proba(X_test)[::,1]

#calculate AUC of model
auc = round( metrics.roc_auc_score(y_test, logPredict), 4 ) 
print("AUC for logistic regression is: ", auc)

## Linear Regression
AUC for linear regression is:  0.7838

In [9]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

#Fit and predict:
lrModel = LinearRegression()
lrModel.fit(X_train, y_train)
lrPredict = lrModel.predict(X_test)

# plt.scatter(y_test, predictions)
plt.hist(y_test - lrPredict)

#Linear Metrics:
auc = round( metrics.roc_auc_score(y_test, lrPredict), 4 ) 
r2 = r2_score(y_test, lrPredict).round(4) 
print("AUC for linear regression is: ", auc)
print("Linear regression r2 score: ", r2)

#CROSS VALIDATE TEST RESULTS:
lr_score = lrModel.score(X_test, y_test).round(4)  # train test 
print("Linear Accuracy: ", lr_score)
lr_cv = cross_validate(lrModel, X, y, cv = 5, scoring= 'r2')
lr_cvMean=lr_cv['test_score'].mean().round(4)
print(lr_cvMean, " linear regression cross validate mean")

def linearReports():
    print(model.coef_)    
    print(model.intercept_)
    print(classification_report(y_test_data, lrPredict))
    print(confusion_matrix(y_test_data, lrPredict))
    metrics.mean_absolute_error(y_test, lrPredict)
    np.sqrt(metrics.mean_squared_error(y_test, lrPredict))

## Decision Tree
The best max_leaf_node is 250.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

#FIND best_tree_size LEAF NODES:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=42)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

def calcLeaf():
    candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
    maeDic={} #dictionary  key=leaf  mae=value
    for leaf in candidate_max_leaf_nodes:
        mae=get_mae(leaf, X_train, X_test, y_train, y_test)
        maeDic[leaf]=mae

    best_tree_size = sorted(maeDic, key=lambda x : maeDic[x])[0]
    print(best_tree_size, " best_tree_size")

best_tree_size=250
    
#MAKE PREDICTION:
tree = DecisionTreeRegressor(max_leaf_nodes=best_tree_size, random_state=42)
tree.fit(X, y)
y_pred = tree.predict(X_test)

#AUC and r2 metric:
treeR2 = r2_score(y_test, y_pred).round(4)
treeAUC = round( metrics.roc_auc_score(y_test, y_pred), 4 ) 
print("AUC for decision tree is: ", treeAUC)

def printReports(y_test, y_pred):
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))
    
    #Mean Sqaured Error:
    treeMSE=np.sqrt(metrics.mean_squared_error(y_test, y_pred))
    print(round(treeMSE, 4), " is tree MSE ")

## Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

#Check for Error and find Best n_estimators:
def checkMAE():
    print("Starting MAE:")
    dMAE={} #dictionary of n_estimators as key and MAE as value:
    for n in range(2, 500, 100):
        forest = RandomForestRegressor(n_estimators=n, random_state = 0)
        forest.fit(X_train, y_train)
        y_pred = forest.predict(X_test)
        MAE=metrics.mean_absolute_error(y_test, y_pred).round(2)
        dMAE[n]=MAE
        print("n_estimates: ", n,  '  Mean Absolute Error:', MAE)

    dMAE=sorted(((v, k) for k, v in dMAE.items()), reverse=False)
    print(dMAE)
#checkMAE() #turn function on or off by uncommenting

def forest():
    num=10
    forest = RandomForestRegressor(n_estimators=num, random_state = 0)
    forest.fit(X_train, y_train)
    y_pred = forest.predict(X_test)

    #Print Metrics:
    forest_r2 = r2_score(y_test, y_pred).round(4)  
    forest_auc = round( metrics.roc_auc_score(y_test, y_pred), 4 ) 
    print("Random forest AUC: ", forest_auc) 
    print("Random forest r2: ", forest_r2)

def forestReports():
    mae=metrics.mean_absolute_error(y_test, y_pred).round(2)
    print("Random forest MAE: ", mae)
    
forest()

# Test Dataset
Here we will make final predictions for the test dataset.

In [ ]:
testDF=pd.read_csv(test)
testDF.head()

## Feature Engineer Test Dataset

In [ ]:
features=list(X.columns)
testDF=testDF[features]
testDF=testDF.fillna(testDF.mean())
testDF.head() #5 rows × 105 columns

## Create Final Predictions

In [ ]:
test_predictions = tree.predict(testDF).round(1)
test_predictions=np.where(test_predictions<0, 0, test_predictions)
ID=testDF['ID']
tupleData = list(zip(ID, test_predictions))
output = pd.DataFrame(tupleData, columns = ['ID', 'TARGET'])
print(output.shape)
output.head()

## My Final Summission

In [ ]:
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

## References
1. https://www.kaggle.com/rahulanand0070/feature-selection
2. https://www.kaggle.com/solegalli/feature-selection-with-feature-engine